In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
# from code.get_args import get_parser
# import get_args
from attn.model import TermModel
import json
import torch
import torch.nn as nn
from transformers import BertTokenizerFast
import torch.utils.data as td
# import lac
# from LAC import LAC



# args = parser.parse_args(['--train_path', '../huawei_dataset/pc.txt','--valid_path','../huawei_dataset/medicine.txt'])


    # parser.add_argument('--sample_probs_choice', type=int, default=1)
    # parser.add_argument('--neg_ratio_choice', type=int, default=1 )
    # parser.add_argument('--context_size', type=int,default=3)
    # parser.add_argument('--log_dir', type=str, default="../logs" )
    # parser.add_argument('--uid', type=str, default="")
    # parser.add_argument('--early_stop_epoch', type=int, default = 10000)
    # parser.add_argument('--use_two_classifier', type=int, default=0)

sam = 0
uniq = 'wzt_alladdc'
epoch = 1

class Args:
    model_name = "hfl/chinese-roberta-wwm-ext"

    # finetuned_ckpt = "ckpts/0914_sam1neg1_contextsize2/0914_sam1neg1_contextsize2_3.pt"
    # context_size = 3

    # wzt_alladdc_neg5sam1_epoch1_split_18

    # 试试epoch4？
    # finetuned_ckpt = f"ckpts/{uniq}_neg5sam{sam}/{uniq}_neg5sam{sam}_epoch{epoch}_split_19.pt"
    finetuned_ckpt = f"/data2/private/wzt/term11_22/term/attn/ckpts/strict_neg_5sam1/strict_neg_5sam1_epoch1_split_19.pt"
    # finetuned_ckpt = f"ckpts/{uniq}_neg5sam{sam}/{uniq}_neg5sam{sam}_epoch{epoch}.pt"
    context_size = 3
    
    use_two_classifier = 0



args = Args()

tk = BertTokenizerFast.from_pretrained(args.model_name)


# train_sents,train_labels = get_sents_labels(args.train_path,args.max_length)
# valid_sents,valid_labels = get_sents_labels(args.valid_path,args.max_length)

# train_pos_ngrams = get_ngrams(train_sents,train_labels,args.max_length)
# valid_pos_ngrams = get_ngrams(valid_sents,valid_labels,args.max_length)

# valid_terms = get_terms(valid_pos_ngrams)

# lac = LAC(mode='seg')
# from IPython import embed; embed(header="true")quit()

# train_ngrams = add_neg(train_pos_ngrams)
# train_sents, train_idxes, train_labels = open_train_ngrams(train_ngrams)


# valid_sents, valid_idxes, valid_labels = get_valid_ngrams(valid_sents)


# train_dataset = SophDatasetNegSampling(train_pos_ngrams, tk,args)
# valid_dataset = SophDatasetNegSampling(valid_pos_ngrams, tk,args)
# train_dataloader = td.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4,collate_fn=SophDatasetNegSampling.collate_fn)
# valid_dataloader = td.DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False, num_workers=4,collate_fn=SophDatasetNegSampling.collate_fn)


# from IPython import embed; embed(header="in soph 48")
# from torch.optim import AdamW




model = TermModel.from_pretrained(args.model_name, num_labels=2)
setattr(model.config, 'args', args)

initialize_from_existing_ckpt = not (args.finetuned_ckpt == "" or args.finetuned_ckpt is None)

if initialize_from_existing_ckpt:
    print("loading finetuned ckpt")
    model.load_state_dict(torch.load(args.finetuned_ckpt))
setattr(model, 'tk', tk)
# model = nn.DataParallel(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
pass

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing TermModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TermModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TermModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TermModel were not initialized from the model checkpoint at hfl/chinese-roberta-wwm-ext and are newly initialized: ['term_key_tra

loading finetuned ckpt


In [2]:
from tqdm import tqdm
def my_ngram(_len, _n):
    res = []
    for i in range(_len):
        for j in range(_n):
            if i+j+1 <= _len:
                res.append((i, i+j+1))
    return res
def get_prefix_len(sent,idx):
    sum = 0
    for i in range(idx):
        sum+=len(sent[i])
    return sum
def have_chinese(strs):
    for _char in strs:
        if '\u4e00' <= _char <= '\u9fa5':
            return True
    return False
N=8
global_dict  ={}
lines_cnt = 0

path = '/home/wangzhitong/data2/strict_sents.txt'
with open(path,'r') as f:
    for line in tqdm(f):
        lines_cnt+=1

with open(path,'r') as f:
    for i,line in enumerate(tqdm(f,total=lines_cnt)):
        line = line.strip()
        line = ''.join(line.split())
        n_gram_idxes = my_ngram(len(line),N)
        for idx in n_gram_idxes:
            if idx[0]!=idx[1]-1:
                char_idx = get_prefix_len(line,idx[0]),get_prefix_len(line,idx[1])-1
                n_gram = ''.join(line[idx[0]:idx[1]])
                if have_chinese(n_gram):
                    global_dict[n_gram] = global_dict.get(n_gram,[])
                    if global_dict[n_gram]!=[1]:
                        global_dict[n_gram].append((line,i,char_idx))
all_ngrams = list(global_dict.keys())

with open(path,'r') as f:
       data = f.readlines()
infer_sents = [list(''.join(item.strip().split())) for item in data]
from tqdm import tqdm
global_set = list(global_dict.keys())
encoded_glb_dict = {}
for g in tqdm(global_set):
    encoded = ''.join(tk.tokenize(g))
    encoded_glb_dict[encoded] = g

10000it [00:00, 557227.09it/s]
100%|██████████| 2222012/2222012 [01:40<00:00, 22122.42it/s]


In [3]:
key_terms = []
with open('/home/wangzhitong/data2/strict_terms.txt','r') as f:
    # terms = f.readlines()
    for line in f:
        line = line.strip()
        key_terms.append(line)
key_terms = set(key_terms)

In [ ]:
key_terms = set(['HAS-A关系', 'IS-A关系', '编程', '超类', '抽象', '抽象数据类型', '对象', '方法', '封装', '格式化运算符', '格式化字符串', '过程抽象', '继承', '继承层次结构', '接口', '可计算', '可修改性', '类', '列表', '列表解析式', '模拟',  '数据抽象', '数据结构', '数据类型', '算法', '提示符', '异常', '真值表', '子类', '字典', '字符串', '大记法', '对数', '对数线性', '蛮力法', '平方',  '清点法', '时间复杂度', '数量级', '线性', '指数', '最坏情况',  '遍历链表', '队列', '后序', '回文', '节点', '链表', '列表', '模拟', '匹配括号', '前序', '数据变量', '双端队列', '头部', '完全括号', '线性数据结构', '优先级', '栈', '中序', '递归', '递归调用', '动态规划', '分形', '栈帧', '步长', '槽', '插入排序', '冲突', '冲突处理', '短冒泡', '二分搜索', '分割点', '分区', '归并', '归并排序', '基准值', '聚集', '开放定址法', '快速排序', '链接法', '冒泡排序', '平方取中法', '平方探测', '三数取中法', '散列', '散列表', '散列函数', '顺序搜索', '完美散列函数', '希尔排序', '线性探测', '选择排序', '映射', '载荷因子', '再散列', '折叠法', 'AVL树', '边', '层数', '二叉堆', '二叉树', '二叉搜索树', '父节点', '高度', '根节点', '后继节点', '后序遍历', '节点', '路径', '前序遍历', '树', '完全二叉树', '兄弟节点', '旋转', '叶子节点', '映射', '优先级队列', '中序遍历', '子节点', '子树', '最小堆','最大堆', '边', '代价', '顶点', '环', '宽度优先搜索', '括号特性', '邻接表', '邻接矩阵', '路径', '强连通单元', '权重', '深度优先森林', '深度优先搜索', '生成树', '拓扑排序', '无环图', '无控制泛滥法', '有环图', '有向图', '有向无环图', '最短路径', 'DFA图', 'DNA串', 'KMP图', 'RSA算法', '八叉树', '层数', '抖动', '公钥加密', '均摊分析', '量化', '确定有限状态自动机', '塔', '跳表', '像素', '映射', '子串', '字典','信息隐藏'])
out = {'DFA','DFS','BFS','KMP','DAG','FIFO','LIFO','self','独立于实现','浅相等','深相等','普通情况','基本情况','堆的有序性','相邻'}
# key_terms = key_terms.union(out)
key_terms = set([item for item in key_terms if item in global_dict])
key_terms = set([item for item in key_terms if len(global_dict[item])>2])
print(key_terms)

In [ ]:
from string import punctuation as enpunc
from zhon.hanzi import punctuation as zhpunc
punc = set(enpunc+zhpunc)
for item in infer_sents:
    # print(item)
    if item[-1] not in punc:
        item.append('。')
    # print(item)
    # break
# infer_text = [''.join(item) for item in infer_sents]
# infer_text

In [4]:

smt = 2
res=[]
for sent in tqdm(infer_sents):
    sent = sent[:128]
    out = model.inference_one_sent(''.join(sent), top_k = -1, verbose=False,sm=True,smt=smt)
    res.append(out)



100%|██████████| 10000/10000 [04:09<00:00, 40.01it/s]


In [ ]:
sent = '轻小说是一种具有高效地将故事内容传达给读者的通俗的写作手法'
out = model.inference_one_sent(sent, top_k = 100, verbose=False,sm=True)
out

In [ ]:
def exe_format(res):
    new_res = []
    for item in tqdm(res):
        new_item = []
        for jtem in item:
            print(jtem)
            new_item.append((''.join(jtem[0][0]),jtem[0][1].cpu().numpy().tolist(),format(jtem[2],'.4f')))
        break
    return new_res
exe_format(res)

In [ ]:


def found_term_by_count_threshold(prob_thres=0, thres=0):
    potential_ngram_count = {}
    for i,sent_ngram in enumerate(tqdm(res)):
        for each_ngram in sent_ngram:
            _word, prob = each_ngram
            _word = ''.join(_word)
            if _word in encoded_glb_dict:
                word = encoded_glb_dict[_word]
                # if word=='' or word=='信息隐藏':
                #     print(word,''.join(infer_sents[i][:256]))
                #     print(prob)
                if prob>prob_thres:
                    # add_score = prob
                    add_score = 1
                    # if prob>1:
                    
                    # # elif prob>1:
                    #     add_score = 2
                    # elif prob > 0.5:
                    #     add_score = 1
                    # else:
                    #     add_score = 0.5

                    if word not in potential_ngram_count:
                        potential_ngram_count[word] = add_score#prob
                    else:
                        potential_ngram_count[word] += add_score#prob
                
    extracted_terms = []
    for key in potential_ngram_count:
        if potential_ngram_count[key] >= min(thres, len(global_dict[key][0])):
            extracted_terms.append(key)
    print(len(extracted_terms))
    extracted_terms = set(extracted_terms)
    intersect = extracted_terms.intersection(key_terms)
    diff_rec = key_terms.difference(extracted_terms)
    diff_prec = extracted_terms.difference(key_terms)
    int_len = len(intersect)
    ext_len = len(extracted_terms)

    print(int_len)
    print(len(key_terms))
    print('p:',int_len/ext_len)
    print('r:',int_len/len(key_terms))
    print(diff_rec)
    print(diff_prec)
    return extracted_terms
    

extracted_terms =  found_term_by_count_threshold(0, 2)


In [ ]:
num = set(str(i) for i in range(10))
num
num_extracted = []
new_extracted = []
for item in extracted_terms:
    # if set(item).intersection(num):
    if '图' in item or '代码清单' in item or '表' in item:
        num_extracted.append(item)
    else:
        new_extracted.append(item)
intersect =set(new_extracted).intersection(key_terms)
int_len = len(intersect)
ext_len = len(new_extracted)
print('p:',int_len/ext_len)
print('r:',int_len/len(key_terms))
diff_prec = set(new_extracted).difference(key_terms)
print(diff_prec)

In [22]:
seet = {}
# seet = {'浅相等','深相等'}
# seet = {'日常生活','这一过程'}
# seet = {'可修改性', '信息隐藏', '堆的有序性', 'HAS-A关系', '相邻', '记法', '最短路径', '前序'}
# seet = {'相邻', '代价'}
# seet= {'前序', '最短路径', '信息隐藏'}
# seet = {'着力点'}
# seet = {'reddit', '浓硫酸', '豨科', '徐惠滋', '李秀英', '构型保持', '边防检查总站', '蓟', '萨仁高娃', '王文惠', '台中市政府', '吉剌思巴监藏巴藏卜', '天津经济技术开发区'}
# seet = {'支气管', '唐宁', '楼文英', '深化党和国家机构改革', '丝织物', '洛追坚赞', '王延康', '冯玉忠', '库尔班江·买买提', '杨澄宇', '中华人民共和国公安部',  '杨仁争', '国家综合配套改革试验区'}
# seet = {'浓硫酸','边防检查总站','台中市政府','地理学界','合同','声波','支气管'}
seet ={'浓硫酸','浓硝酸','稀硫酸','稀硝酸','高氯酸','浓磷酸','稀磷酸','浓盐酸','稀盐酸','稀醋酸','浓醋酸','浓溶液','浓咖啡','稀奶油'}
# seet = {'红玫瑰','白玫瑰'}
seel = []
print_term_dict = {}
occur_words = []
saveres = []
def see_term_by_count_threshold(prob_thres=0, thres=0,mean_thres=0):
    
    potential_ngram_count = {}
    for i,sent_ngram in enumerate(tqdm(res)):
        for each_ngram in sent_ngram:
            (_word,_),prob = each_ngram
            _word = ''.join(_word)
            if _word in encoded_glb_dict:
                word = encoded_glb_dict[_word]
                occur_words.append(word)
                # sent = ''.join(infer_sents[i][:256])
                # if word in print_term_dict:
                #     print_term_dict[word].append([i,prob])
                # else:
                #     print_term_dict[word] = [[i,prob]]
                saveres.append([word,i,prob])
                if word in seet:
                    seel.append([word,''.join(infer_sents[i][:256]),prob])
                    # print(word,''.join(infer_sents[i][:256]))
                    # print(prob)
                add_score = 0
                if prob>prob_thres:
                    add_score = 1

                if word not in potential_ngram_count:
                    potential_ngram_count[word] = [add_score,1,prob]#prob
                else:
                    potential_ngram_count[word][0] += add_score#prob
                    potential_ngram_count[word][1] +=1
                    potential_ngram_count[word][2] +=prob        

    extracted_terms = []
    for key in potential_ngram_count:
        # if potential_ngram_count[key][0] >= min(thres, len(global_dict[key])):
        if potential_ngram_count[key][0] >= thres:
            # if potential_ngram_count[key][2]/potential_ngram_count[key][1] >= mean_thres:
                extracted_terms.append(key)
    print(len(extracted_terms))
    extracted_terms = set(extracted_terms)
    intersect = extracted_terms.intersection(key_terms)
    diff_rec = key_terms.difference(extracted_terms)
    diff_prec = extracted_terms.difference(key_terms)
    int_len = len(intersect)
    ext_len = len(extracted_terms)

    print(int_len)
    print(len(key_terms))
    print('p:',int_len/ext_len)
    print('r:',int_len/len(key_terms))
    # print(diff_prec)
    return extracted_terms,potential_ngram_count,diff_prec
    

extracted_terms,count_dic,diff_prec =  see_term_by_count_threshold(0.5, 2)
num_extracted = []
new_extracted = []
import re 
for item in extracted_terms:
    # if set(item).intersection(num):
    # 图 表 代码清单
    if re.search(r'图\d+\-\d+',item) or re.search(r'表\d+\-\d+',item) or re.search(r'代码清单\d+\-\d+',item) or re.search(r'\d+\.\d+(\.\d)?',item):
    # if '图' in item or '代码清单' in item or '表' in item:
        num_extracted.append(item)
    else:
        new_extracted.append(item)
print('p:',len(key_terms.intersection(new_extracted))/len(new_extracted))
print('r:',len(key_terms.intersection(new_extracted))/len(key_terms))

100%|██████████| 10000/10000 [00:10<00:00, 931.62it/s]


8418
7522
1661229
p: 0.8935614160133049
r: 0.004527972964594285
p: 0.8937737642585551
r: 0.004527972964594285


In [87]:
tmp = ['其他']
tmp =set(tmp)
sent = ''
for item in tqdm(saveres):
    word = item[0]
    i = item[1]
    prob = item[2]
    if item[0] in tmp:
        print(word,''.join(infer_sents[i][:256]),prob)
        sent = ''.join(infer_sents[i][:256])

 14%|█▍        | 346116/2395389 [00:00<00:01, 1577415.75it/s]

其他 主要文章：核合成和恒星核合成虽然它是在大霹雳中合成的，但是锂（和铍及硼）在宇宙中的含量明显低于其他元素。 0.006042726
其他 泪水冲走了其他动物和雀鸟；一时间，爱丽丝已被一群动物包围。 0.0075329747
其他 在与摩城签约后的一年半，组合于1967年转至摩城旗下的灵魂唱片（soulrecords），并录制了〈〉以及其他随之而来的热门歌曲，包括〈〉、〈friendshiptrain〉、〈〉和〈〉。 0.007156176
其他 从6月30日起瓦德和其他被捕者开始绝食，他们于7月4日被释放，8月30日其他的所有控告也全部被放弃。 0.0064896774
其他 从6月30日起瓦德和其他被捕者开始绝食，他们于7月4日被释放，8月30日其他的所有控告也全部被放弃。 0.004020675
其他 在尾巴底部的两侧各有一块白色的毛，与狒狒和其他猴子的屁股非常相像 0.012619244
其他 阿根廷同时与南美其他国相比拥有较良好的工业基础，自然资源也非常丰富，阿根廷受益于丰富的自然资源，高度受教育的人口，出口导向的农业部门，多样的工业基础。 0.007055091
其他 里贝克的律师声称麦当劳的咖啡送到消费者手中的时候是华氏180-190度（摄氏82-88度），其他餐厅的咖啡一般在华氏135-140度（摄氏57-60度），麦当劳的咖啡更热，容易造成烫伤，是产品缺陷。 0.007788985
其他 在慈运理的领导下，信仰新教的苏黎世州与市同其他新教州组成抵抗联盟“基督教公民联盟”，其中还包括康斯坦茨和斯特拉斯堡。 0.0032671294
其他 黎巴嫩不同于其他中东国家以穆斯林为主，黎巴嫩宗教分布比较类似于波赫和阿尔巴尼亚，都为穆斯林和基督徒各占一半的比例。 0.005233209
其他 与其他非洲国家一致，足球是科特迪瓦的第一运动。 0.0027269053
其他 虽然他和诺勒斯与其他指挥官同样有责任，但他将罪过推给诺勒斯来回避罪责。 0.0025935709
其他 他们一入宇宙，就着手依埃努的大乐章旋律建造世界，维拉时代开始，但米尔寇向其他维拉宣布他的主权，并和他的追随者对阿尔达大加破坏，扰乱已成形的部分，并转作私有用途。 0.010880384
其他 庞培则在色萨利与其他贵族共和派成员汇合，在后者的再三催促下，庞培军在法萨卢斯附近与凯撒军正面冲突。 

 27%|██▋       | 654160/2395389 [00:00<00:01, 1390949.73it/s]

其他 运河事件后，英国决定将其全球军事部署退防至苏伊士运河以西，放弃在全球其他地区单方面出兵干涉的能力。 0.0039550955
其他 一般在游戏开始前，玩家必须先构筑自己的牌组好参加游戏：在任何时候，每张牌只能位于下列其中一个“区域”，其中战场和堆叠为共用区域，其他区域则是每位玩家各自独立。 0.011551445
其他 大禹治水的过程记载在《尚书·禹贡》，当时的已知世界被划分为九州，洛阳属禹贡九州的豫州，居其他八州之中央。 0.016279407
其他 檀是相当古老的中文词汇，《诗经》魏风有《伐檀》三章：估计成书于东周或更早的《诗经》，一般认为收录的是中国3000年左右以前的黄河中、下游的民歌，当时中国此一区域的檀树，就只有青檀一种，其他分布在南方的黄檀等，其所在区域可能尚非中国版图，所以一般推断此处用以制辐、制轮的檀，就是青檀。 0.009133799
其他 《综艺》认为影片在很多层面上都具有开创性，尤其是它在主角性取向之外与其他青春片的惊人相似。 0.0038521895
其他 在其他方向上，如果我们有对"ax=b的另一个解z，则"a"(z−y)="az−"ay=b−b=0。 0.011731647
其他 除格林中心以外，洪堡大学还有众多专业图书馆以及自然科学图书馆，在格林中心启用之后，其他图书馆原址未动。 0.0032483942
其他 其他目的地还包括尼泊尔、摩洛哥、斐济、厄瓜多尔和萨尔瓦多。 0.002380249
其他 一些成员认为他患上神经衰弱或脑动脉硬化，然而其他成员相信他感染梅毒。 0.002850048
其他 除了触觉敏感外，它们的其他感官没有发育。 0.00983266
其他 他还以拨款和其他方式支持一家德国文化中心。 0.0070168106
其他 尼尔森一直拥有《公告牌》杂志到2009年，在那年，《公告牌》与其他7本出版物被出售给易伍全球传媒控股公司。 0.005157773
其他 其相对分子质量小，相对其他净气物质效率更高，1克的无水氢氧化锂可以吸收450cm的二氧化碳气体。 0.006037112
其他 在这个人口不到9万的小城，就有800家生产丝绸和其他高级纺织品的公司，业务涉及生产、印花、染色、设计和销售。 0.007161432
其他 这些学校因为有华社的大力支持，使得教育质量比其他源流学校来的高，也因此吸引了许多非华裔华小

 33%|███▎      | 794034/2395389 [00:00<00:01, 1243915.52it/s]

...

其他 和低温下呈反铁磁性和/或铁磁性的其他稀土金属不同，镱在1.0k以上具有顺磁性。 0.0077377963
其他 其他的得分等级分为半胜（waza-ari）、有效（yuko）。 0.0060400493
其他 久华洋行专事收购铜板及其他铜器，为了使铜更快更多地流入日本，日军还发放收购铜货证，持该凭证者均有权收购铜板、铜器等，然后转卖给日本的洋行。 0.009406545
其他 1943年，《我们妇女》在雅加达首次上演，同年这套剧目还在雅加达和爪哇岛其他地方上演过很多次。 0.0010801496
其他 这些管理器通常还提供一个窗口，显示最近的操作记录，允许用户选择更早的副本，然后编辑它们、更改它们的格式、搜索它们，以及其他高级功能。 0.011225598
其他 素有超马女王称号的她，平时除了在芦洲慢跑社团之外，也会把自己的超马心得分享给其他跑者，期望他们能够健康跑旅，当然也有很多超马跑者跟她请益！ 0.008308389
其他 一般情况下，供给定律描述的是：假设其他因素不变，当一件物品的相对价格上升时，其供给量会上升，反之亦然。 0.008847109
其他 8岁时，碧昂丝在一个女子团体的试音会上遇见拉塔薇亚·罗伯森（lataviaroberson），两人与碧昂丝的好友凯莉·罗兰，以及其他团员共同组成一个表演rap和舞蹈的团体，团体原名为“girl'styme”，之后团员人数减到6名。 0.0046668393
其他 其他语言中最常见的是中文（26.7%），其次为广东话（20.4）、希腊语（3.7%）以及阿拉伯语（3.6%）。 0.0049262564
其他 经过这次战略挫折，腓特烈认识到，因为普鲁士本身人口有限，军中很多是德意志其他邦国的雇佣兵，这样的军队，必须要有铁的纪律才能凝聚成战斗力，而且绝不能分散使用。 0.0036221019
其他 它可使usb设备，例如播放器或手机，从usb周边设备变为usb主机，与其他usb设备连接通信。 0.002967615
其他 9月下旬，漫威选择赵婷执导这部电影，由于在会面中对赵留有良好印象，漫威担心她会选择其他大制片厂商的电影项目，双方很快签订了导演协议。 0.0044069467
其他 dicom的档案是由标准化且自由型式的开头再加上一连串的影像数据，单一个dicom的物件只包含一张影像，但是此影像可能会有多个套图

 98%|█████████▊| 2335856/2395389 [00:02<00:00, 840110.06it/s]

其他 2012年6月大选中奥尼尔再次当选议员，奥尼尔的与其他政党组成联合政府，在8月3日组成新一届政府，奥尼尔继续担任总理。 0.0031439252
其他 pascal没有main或是其他相类似的关键字。 0.0051476783
其他 eia-485上最常使用的协定都是属于，不过也有其他的通讯，例如（ssi）。 0.0073654703
其他 猫剧的角色介绍，此剧的一大特色是舞台上的每只猫都有自己的名称和性格，这在其他的音乐剧并不常见。 0.010322793
其他 《孙子兵法》注家杜牧认为，曹操将八十二篇孙子兵法删节为十三篇；但其他注家认为十三篇出自孙武本人，不是曹操删节的结果。 0.0077213612
其他 在吴全病逝后，残存的移民也迁到其他地方去，城池也因而荒芜。 0.0035504568
其他 1980年代，加拿大公园管理局将很多公园的设施私有化，如高尔夫球场等，并增加了使用其他设施的费用，以削减预算。 0.0069341515
其他 同时，领英没有为中文用户提供某些重要功能，比如创建或加入群组，或是张贴长篇文章，而其他地方的用户就可以开展公开讨论、组建社区。 0.008136291
其他 其后外尔向爱因斯坦等其他物理学家广泛讨论了这个理论在物理上的正确性，但最终这个理论被证明是在物理上不合理的。 0.0031362402
其他 1979年5月10日，接掌新约教会的洪以利亚正式在锡安山定居，同工（牧师）反对他这样做，认为他这样是放下了新约教会的文字工作、以及其他地方教会；洪以利亚反驳指，是神要求他住在锡安山，有如摩西留在西奈山一样，离开锡安山就像离开神，此后同工的态度有所转变。 0.00602942
其他 中建电讯更向法院申请禁制令，全面封杀刘德华为其他公司做唱片、接拍影视和广告等演出机会，等于全面封杀刘的演艺事业，结果禁制令成功，使得刘德华在禁制令生效期间估计损失收入上亿元。 0.0065243165
其他 北部山区则主要依赖所经营的客运路线，其他包括和在辖内也都有经营客运路线。 0.005930889


100%|██████████| 2395389/2395389 [00:02<00:00, 985130.43it/s]

其他 不过这样它需要保护猎物，免受其他掠食者抢夺，但其保护策略就不得面知。 0.0067132274


In [90]:

sent='其他地方的用户可以开展公开讨论。'
out = model.inference_one_sent(sent, top_k = 100, verbose=False,sm=True)
for item in out:
    ngram = ''.join(item[0][0])
    prob = format(item[1],'.3f')
    print(f'{ngram},{prob}')
# out

用户,0.599
讨论,0.208
地方,0.103
公开讨论,0.035
公开,0.035
公,0.005
论,0.003
用,0.003
方,0.002
展公开,0.002
展公开讨论,0.002
开展公开讨论,0.002
公开讨,0.002
地,0.002
展,0.001
开展,0.001
开展公开,0.001
展公开讨,0.001
开,0.001
展公,0.001
开展公开讨,0.000
开展公,0.000
讨论。,0.000
户,0.000
开讨,0.000
开讨论,0.000
公开讨论。,0.000
讨,0.000
开展公开讨论。,0.000
其他地方的用户,0.000
其他地方,0.000
其他地方的用户可以开展公开讨论,0.000
展公开讨论。,0.000
用户可以开展公开讨论,0.000
其他地方的用,0.000
其他地方的,0.000
其他地,0.000
其他,0.000
可以开展公开讨论,0.000
用户可,0.000
其他地方的用户可以开展,0.000
地方的用户,0.000
其他地方的用户可,0.000
他地方,0.000
其他地方的用户可以,0.000
户可以开展公开讨论,0.000
用户可以开展,0.000
地方的用户可以开展公开讨论,0.000
户可,0.000
地方的,0.000
可以开展,0.000
其他地方的用户可以开,0.000
其,0.000
用户可以,0.000
地方的用,0.000
户可以开展,0.000
用户可以开,0.000
他地方的用户,0.000
。,0.000
地方的用户可以开展,0.000
他地方的用户可以开展公开讨论,0.000
可以开,0.000
地方的用户可以,0.000
的用户可以开展公开讨论,0.000
可以,0.000
可,0.000
他地,0.000
地方的用户可,0.000
用户可以开展公开讨论。,0.000
户可以,0.000
户可以开,0.000
开,0.000
以开展公开讨论,0.000
他地方的用,0.000
他地方的,0.000
以开展,0.000
地方的用户可以开,0.000
他,0.000
的用户,0.000
他地方的用户可以开展,0.000
地方的用户可以开展公开讨论。,0.000
的用户可以开展,0.000
他地方的用户可以,0.000
他地方的用户可,0.000
方的用户可

In [70]:
# occur_words = set(occur_words)
re_patterns = [
    r'^\d+日$',
    r'^\d+月\d*$',
    r'^\d+年$',
    r'^\d+年\d+月$',
    r'^\d+年\d+月\d+日$',
    r'^\d+月\d+日$',
    r'^[公元]*\d+世纪$',
    r'^在\d+年$',
    r'^第\d+$',
    r'^\d+年代$'
    r'^月\d+$'
]
import re
def have_pat(term):
    for pattern in re_patterns:
        if re.search(pattern, term):
            # print('yes')
            return True
    return False
from collections import Counter
occur_cnt = Counter(occur_words)
new_key_terms = set([item for item in key_terms if item in occur_cnt and occur_cnt[item]>1 and len(item)>2])
print('p:',len(new_key_terms.intersection(new_extracted))/len(new_extracted))
print('r:',len(new_key_terms.intersection(new_extracted))/len(new_key_terms))
# 找汉字
import re
hanzire = re.compile(u'[\u4e00-\u9fa5]')
# hanzire.search(target)
noyear = set([item for item in new_key_terms if  hanzire.search(item) and not have_pat(item)])
diff_prec = noyear.difference(new_extracted)
diff_prec
# print(diff_prec)
import random
print(random.sample(diff_prec,200))

p: 0.4152804182509506
r: 0.628032345013477
['24小时', '看起来', '革命军', '联邦铁路', '最适合', '菲尔德', '大天王', '交通线', '布劳恩', '同年8月', '留学生', '纳斯吉', '灯笼鱼', '县学生', '罗伊特', '管理员', '尼特拉', '开发商', '神学院', '电偶极', '威斯特', '台湾社', '布伦特', '德川家', '地杨梅', '混合制', '对台工作', '蓝绿色', '金像奖', '维伦纽夫', '爱达荷', '隔距兰', '内战爆发', '市议员', '分类法', '超级联赛', '最接近', '布哈拉', '西南太平洋', '下半年', '20岁', '伯纳德', '帝国皇帝', '比利亚', '佛得角', '奥利弗', '怎么做', '高尔夫', '布维尔', '电台节目', '集水区', '党领袖', '世纪末', '不定时', '美术家', '霍尔木兹', '见证人', '纽奥良', '纪念地', '塞缪尔', '研究社', '含氧量', '普通法', '有意识', '圣彼得', '中华民国陆', '海军少', '新文化', '人民委员', '澳门甲组', '郡太守', '卡鲁斯', '研究者', '艾森豪', '公安厅', '匈帝国', '三环路', '马克斯', '省人民政府', '卡德罗', '西印度', '皇太后', '西大区', '中央军', '扎波罗热', '锅炉房', '尼斯特', '教育学', '人民共和国', '研究系', '附加赛', '兰伯特', '凯瑟琳', '霍夫曼', '马拉维', '西班牙裔', '10次', '人口普查指定', '400米', '爱荷华', '大布尔卢克', '东南面', '吐鲁番', '第25届', '发行量', '卡尔斯鲁', '大虎山', '司令员', '生于上海', '明星赛', '托马斯', '坦尼斯', '训练场', '东上线', '中央书记处', '裁判所', '宁东站', '长老会', '中央情报', '主教堂', '乌拉尔', '自然地', '京都交通', '第十三届', '警察部队', '多目标', '普林斯', '葡萄园', '米尔纳', '第120师', '格鲁吉', 

/tmp/ipykernel_2924923/1867590202.py:36: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  print(random.sample(diff_prec,200))


In [82]:
from string import punctuation as punc
from zhon.hanzi import punctuation as zh_punc
punc = set(punc+zh_punc)
def have_punc(s,punc = punc):
    for c in s:
        if c in punc:
            return True
    return False


diff_rec  = set(new_extracted).difference(noyear)
# twot = set(two_terms)
diff_rec = [item for item in diff_rec if hanzire.search(item) and not have_pat(item) and len(item)>2]
# diff_rec
print(random.sample(diff_rec,200))

['为害新娘', '石油公司', '签名人', '防炮车', '通讯及监察', '可证明', '街市街市', '国家铁路建设', '枚方町', '销量排行榜', '电视事业', '林柄存', '粉壁岭', '罗马帝国统治', '禾鸭生态', '美国世嘉', '中共中央总书记', '卵二姐', '鱼快乐', '医介辅', '货运营业', '职业技术学院', '钢铁人', '最高人民法院', '后鳃类物种', '军官级', '全体会议', '硕士研究生教育', '国务院副总理', '葛雷佛', '相似度', '埃尔斯特', '人权研究所', '浪淘沙三部曲', '中国双节棍邀请赛', '中央草坪', '赛车手', '广播电视', '主要车站', '歌唱大赛', '勇者斗恶龙', '推力重量', '美国nba', '亚伯拉罕·伊本', '西文化', '舰艇学院', '临时货币', '换乘通道', '儒林戏', '李家窑', '巴士车厂', '主管部门', '蒸汽机车制造厂', '特种部队2', '《圣诞祭》', '秘书室', '缓冲簧', '全自动波箱', '田吉兹', '林克斯市场', '生活环境', '52赫兹', '背景乐', '票选结果', '照明灯', '社会经济', '标准反应焓', '谱分辨率', '南方都市报', '喝茶会', '严重冲突', '文学教授', '圣经书社', '冶金集团', '热带高压脊', 'nba联盟', '荣誉称号', '少年时期', '艾芙琳', '饱和直链脂肪酸', '詹姆斯、丹尼', '新填地街市场', '亚冠小组赛', '雅砻江口', '通讯委员会', '制作动画', '丹尼、汤姆', '常规时间', '原乡人', '“慰安妇”', '攻克北京', '二重唱公演', '郑和舰队', '政治局常委', '《信报》', '萨默赛特', '阿纳斯塔', '人权研究', '女主角奖', '人民检察院', '第三方软件', '绩优教职员工', '数字转换器', '中华成棒', '美国政府', '魔法史', '绝对领域', '客运中心', '马利克·艾亚兹', '填地街市场', 'ldl胆固醇', '香港电影金像奖', '吸虫囊蚴', '荷兰当局', '合约无效', '何文田北', '暹罗王位', '社会经济问题', '剑击队', '民族

In [ ]:
import random
diff_prec = [item for item in diff_prec if '第' not in item]
print(random.sample(diff_prec,len(diff_prec)))

In [ ]:
all_terms = []
with open('/home/wangzhitong/data2/all_train_terms.txt','r') as f:
    for line in f:
        all_terms.append(line.strip())
all_terms = set(all_terms)
allnoyear = set([item for item in all_terms if item in occur_words and '年' not in item and '世纪' not in item and hanzire.search(item) and not have_punc(item)])

In [ ]:
two_terms = []
with open('/home/wangzhitong/data2/all_train_terms.txt','r') as f:
    for line in tqdm(f):
        word = line.strip()
        if len(word )==2:
            two_terms.append(word)

In [ ]:
diff_rec = set(new_extracted).difference(allnoyear)
# print(diff_rec)
print(random.sample(diff_rec,200))

In [ ]:
p = 29.8
r = 95.1
f1 = 2*p*r/(p+r)
print(f1)